In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import random
import time
import tensorflow as tf

In [2]:
n_train = 4640
n_valid = 131

In [3]:
IMG_SIZE = 160

def load(dir, n):
    x_data, y_data = [], []
    for i in range(n):
        image = cv2.imread(dir + "/" + str(i + 1) + ".png")
        r = open(dir + "/" + str(i + 1) + ".txt", "r")
        label = int(r.readline())
        image = tf.cast(image, tf.float32)
        image = (image/127.5) - 1
        x_data.append(image)
        y_data.append(label)
    return x_data, y_data

X_train, Y_train = load("/home/dima/PycharmProjects/Practics/Dataset/train", n_train)
X_valid, Y_valid = load("/home/dima/PycharmProjects/Practics/Dataset/valid", n_valid)
Y_train = tf.cast(Y_train, tf.int32)
Y_valid = tf.cast(Y_valid, tf.int32)
Y_train -= 1
Y_valid -= 1
print(Y_train)
X_train_norm = tf.norm(X_train)
X_valid_norm = tf.norm(X_valid)

#X_train = X_train / X_train_norm
#X_valid = X_valid / X_valid_norm


tf.Tensor([0 0 0 ... 3 3 3], shape=(4640,), dtype=int32)


In [4]:
train = tf.data.Dataset.from_tensor_slices( (X_train, Y_train) )
valid = tf.data.Dataset.from_tensor_slices( (X_valid, Y_valid) )

In [5]:
print(train)
print(valid)

<TensorSliceDataset shapes: ((160, 160, 3), ()), types: (tf.float32, tf.int32)>
<TensorSliceDataset shapes: ((160, 160, 3), ()), types: (tf.float32, tf.int32)>


In [6]:
BATCH_SIZE = 29
SHUFFLE_BUFFER_SIZE = 1000

In [7]:
train_batches = train.batch(BATCH_SIZE)
validation_batches = valid.batch(BATCH_SIZE)
print(train_batches, validation_batches)

<BatchDataset shapes: ((None, 160, 160, 3), (None,)), types: (tf.float32, tf.int32)> <BatchDataset shapes: ((None, 160, 160, 3), (None,)), types: (tf.float32, tf.int32)>


In [8]:
for image_batch, label_batch in train_batches.take(1):
    pass

image_batch.shape

TensorShape([29, 160, 160, 3])

In [9]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)


base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


In [10]:
feature_batch = base_model(image_batch)
print(feature_batch.shape)


(29, 5, 5, 1280)


In [11]:
base_model.trainable = False
base_model.summary()


Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [12]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)


(29, 1280)


In [13]:
import tensorflow.keras as keras

prediction_layer = keras.layers.Dense(4, activation = 'softmax')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)


(29, 4)


In [14]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 4)                 5124      
Total params: 2,263,108
Trainable params: 5,124
Non-trainable params: 2,257,984
_________________________________________________________________


In [15]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [16]:
print(len(model.trainable_variables))
for image_batch, label_batch in validation_batches:
    print(label_batch)

2
tf.Tensor([1 1 3 3 0 3 3 2 1 2 2 1 1 2 1 3 0 1 2 2 3 3 3 0 3 2 2 3 1], shape=(29,), dtype=int32)
tf.Tensor([0 1 1 3 0 1 2 3 3 0 1 1 3 2 1 1 0 2 0 1 1 1 0 1 2 1 0 0 0], shape=(29,), dtype=int32)
tf.Tensor([1 0 2 3 3 0 0 0 3 0 2 0 0 1 1 3 1 1 3 0 1 1 2 0 0 3 2 0 2], shape=(29,), dtype=int32)
tf.Tensor([3 2 2 0 0 2 1 0 0 0 3 1 2 0 3 2 0 0 2 0 3 1 2 2 1 3 2 3 0], shape=(29,), dtype=int32)
tf.Tensor([0 2 3 2 3 1 2 3 2 1 3 3 1 3 2], shape=(15,), dtype=int32)


In [17]:
validation_steps = 5
initial_epochs = 10
model.evaluate(validation_batches, steps = validation_steps)
predictions = model.predict(validation_batches, steps = validation_steps, verbose = 1)
print(predictions)

5/5 [==============================] - 5s 1s/step
[[0.6334113  0.07529165 0.12947232 0.1618247 ]
 [0.75924045 0.07796205 0.06651464 0.09628289]
 [0.47817567 0.25403643 0.10541909 0.16236879]
 [0.7928577  0.04913662 0.05621391 0.10179188]
 [0.8601629  0.03066578 0.05046929 0.05870207]
 [0.5458394  0.04079972 0.2508625  0.1624984 ]
 [0.17126036 0.07174995 0.13467611 0.6223136 ]
 [0.39483064 0.24041751 0.08295697 0.28179488]
 [0.46514407 0.35714033 0.06365591 0.1140597 ]
 [0.3875624  0.12405586 0.14549239 0.34288934]
 [0.4209137  0.23350933 0.07257742 0.27299958]
 [0.41257936 0.36501232 0.10010874 0.12229962]
 [0.639823   0.06132953 0.09290395 0.20594348]
 [0.32293883 0.2512325  0.16167606 0.26415262]
 [0.31368896 0.31983843 0.21774343 0.1487292 ]
 [0.90534985 0.03251551 0.02676104 0.03537355]
 [0.5969581  0.10169344 0.06101881 0.24032968]
 [0.7151536  0.10715634 0.09716859 0.08052146]
 [0.5255007  0.03391467 0.05975471 0.38082996]
 [0.6379049  0.05097071 0.08923279 0.22189152]
 [0.739261

In [18]:
history = model.fit(train_batches,
                    epochs=initial_epochs,
                    validation_data=validation_batches)


Epoch 1/10
160/160 [==============================] - 228s 1s/step - loss: 9.2399 - accuracy: 0.0866 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
160/160 [==============================] - 219s 1s/step - loss: 9.0544 - accuracy: 0.1181 - val_loss: 11.9206 - val_accuracy: 0.9618
Epoch 3/10
160/160 [==============================] - 200s 1s/step - loss: 8.9806 - accuracy: 0.1515 - val_loss: 12.2610 - val_accuracy: 0.9695
Epoch 4/10
160/160 [==============================] - 189s 1s/step - loss: 8.9361 - accuracy: 0.1759 - val_loss: 12.5136 - val_accuracy: 0.9695
Epoch 5/10
160/160 [==============================] - 197s 1s/step - loss: 8.9078 - accuracy: 0.1955 - val_loss: 12.6920 - val_accuracy: 0.9695
Epoch 6/10
160/160 [==============================] - 212s 1s/step - loss: 8.8884 - accuracy: 0.2125 - val_loss: 12.8124 - val_accuracy: 0.9695
Epoch 7/10
160/160 [==============================] - 254s 2s/step - loss: 8.8736 - accuracy: 0.2233 - val_loss: 12.8897 - val_ac

In [19]:
model.evaluate(validation_batches, steps = validation_steps)

5/5 [==============================] - 3s 646ms/step - loss: 12.9705 - accuracy: 0.9695


[12.970525932312011, 0.9694657]

In [20]:
predictions = model.predict(validation_batches, steps = validation_steps, verbose = 1)
print(predictions)

5/5 [==============================] - 3s 666ms/step
[[0.79505587 0.04424897 0.06075171 0.09994356]
 [0.8821059  0.0390464  0.02688823 0.05195947]
 [0.70984524 0.14209285 0.04761766 0.10044423]
 [0.8890985  0.02714297 0.02551422 0.05824422]
 [0.93016803 0.01641449 0.02127501 0.03214242]
 [0.7459172  0.0244295  0.12699173 0.10266153]
 [0.32321236 0.05766563 0.09107932 0.5280427 ]
 [0.61120456 0.15518418 0.04392191 0.18968935]
 [0.60642743 0.26598573 0.03913894 0.08844794]
 [0.6243331  0.07629231 0.07377825 0.22559634]
 [0.6147824  0.15829808 0.04081248 0.18610705]
 [0.60238194 0.25146675 0.0582338  0.08791758]
 [0.8270075  0.02878554 0.03566764 0.10853938]
 [0.50922406 0.18930861 0.09736226 0.20410506]
 [0.55360395 0.21273428 0.12103492 0.11262688]
 [0.9600649  0.01436967 0.00915883 0.01640654]
 [0.74834454 0.06376921 0.0312794  0.15660684]
 [0.8578485  0.05703763 0.04068063 0.04443315]
 [0.6863902  0.02256387 0.03275406 0.2582919 ]
 [0.8153684  0.02684565 0.03756382 0.12022212]
 [0.885